In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
!pip install ipysheet
import ipysheet
import pandas as pd
from ipywidgets import GridspecLayout, FloatText, Button, Output, HTML, Text, VBox, HBox
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import linprog

In [ ]:
print("ANÁLISIS DE TEORÍA DE LOS JUEGOS")
print("="*50)
print("\nConfiguración de la matriz de pagos:")
print("- Jugador Usuario: FILAS")
print("- Jugador Contrincante: COLUMNAS")
print("="*50)

modo_widget = widgets.ToggleButtons(
    options=[('Suma Cero (Competitivo)', 'suma_cero'), ('Suma No Cero (General)', 'no_suma_cero')],
    description='Modo:',
    button_style='',
    style={'description_width': 'initial'}
)

filas_widget = widgets.IntSlider(value=2, min=2, max=10, step=1, description='Filas:', continuous_update=False)
columnas_widget = widgets.IntSlider(value=2, min=2, max=10, step=1, description='Columnas:', continuous_update=False)
boton = widgets.Button(description='Confirmar Configuración', button_style='success')
output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        global filas, columnas, modo_juego
        filas = filas_widget.value
        columnas = columnas_widget.value
        modo_juego = modo_widget.value
        print(f"\nMatriz de {filas}x{columnas} creada exitosamente.")
        print(f"Modo de juego: {'Suma Cero' if modo_juego == 'suma_cero' else 'Suma No Cero'}")
        print(f"Jugador Usuario: {filas} estrategias (filas)")
        print(f"Jugador Contrincante: {columnas} estrategias (columnas)")

boton.on_click(on_button_click)

display(modo_widget, filas_widget, columnas_widget, boton, output)

In [ ]:
print(f"\nMatriz de pagos {filas}x{columnas}")
print("(1) Nombre las estrategias")
print("(2) Ingrese los valores en las celdas\n")

nombres_filas = []
print("Nombres de estrategias del Jugador Usuario (Filas):")
for i in range(filas):
    nombre = Text(value=f'Estrategia F{i+1}', description=f'Fila {i+1}:', layout={'width': '300px'})
    nombres_filas.append(nombre)

nombres_columnas = []
print("\nNombres de estrategias del Jugador Contrincante (Columnas):")
for j in range(columnas):
    nombre = Text(value=f'Estrategia C{j+1}', description=f'Col {j+1}:', layout={'width': '300px'})
    nombres_columnas.append(nombre)

boton_actualizar = Button(description='Actualizar Nombres', button_style='warning')
output_grid = Output()

def actualizar_grid(b=None):
    with output_grid:
        clear_output()
        
        if modo_juego == 'suma_cero':
            print("Matriz de Pagos (Jugador Usuario)")
            print("Lo que gana Usuario es lo que pierde Contrincante\n")
            
            grid = GridspecLayout(filas+1, columnas+1)
            grid[0, 0] = HTML(value="<b></b>")
            for j in range(columnas):
                grid[0, j+1] = HTML(value=f"<b>{nombres_columnas[j].value}</b>", layout={'width': '100px'})
            for i in range(filas):
                grid[i+1, 0] = HTML(value=f"<b>{nombres_filas[i].value}</b>", layout={'width': '150px'})
            
            global cells_matriz
            cells_matriz = []
            for i in range(filas):
                fila = []
                for j in range(columnas):
                    cell = FloatText(value=0.0, layout={'width': '100px'})
                    grid[i+1, j+1] = cell
                    fila.append(cell)
                cells_matriz.append(fila)
            
            display(grid)
        else:
            print("Matrices de Pagos Bimatricial")
            print("Ingrese (Pago Usuario, Pago Contrincante) en cada celda\n")
            
            grid = GridspecLayout(filas+1, columnas+1)
            grid[0, 0] = HTML(value="<b></b>")
            for j in range(columnas):
                grid[0, j+1] = HTML(value=f"<b>{nombres_columnas[j].value}</b>", layout={'width': '200px'})
            for i in range(filas):
                grid[i+1, 0] = HTML(value=f"<b>{nombres_filas[i].value}</b>", layout={'width': '150px'})
            
            global cells_matriz_j1, cells_matriz_j2
            cells_matriz_j1 = []
            cells_matriz_j2 = []
            for i in range(filas):
                fila_j1 = []
                fila_j2 = []
                for j in range(columnas):
                    container = HBox([
                        FloatText(value=0.0, description='J1:', layout={'width': '80px'}, style={'description_width': '25px'}),
                        FloatText(value=0.0, description='J2:', layout={'width': '80px'}, style={'description_width': '25px'})
                    ])
                    grid[i+1, j+1] = container
                    fila_j1.append(container.children[0])
                    fila_j2.append(container.children[1])
                cells_matriz_j1.append(fila_j1)
                cells_matriz_j2.append(fila_j2)
            
            display(grid)

boton_actualizar.on_click(actualizar_grid)

boton_confirmar = Button(description='Confirmar Valores', button_style='info')
output_matriz = Output()

def on_confirmar_click(b):
    with output_matriz:
        clear_output()
        global matriz, matriz_j1, matriz_j2, nombres_estrategias_filas, nombres_estrategias_columnas
        nombres_estrategias_filas = [n.value for n in nombres_filas]
        nombres_estrategias_columnas = [n.value for n in nombres_columnas]
        
        if modo_juego == 'suma_cero':
            matriz = np.zeros((filas, columnas))
            for i in range(filas):
                for j in range(columnas):
                    matriz[i][j] = cells_matriz[i][j].value
            
            print("\nMatriz de pagos confirmada (Suma Cero):")
            df = pd.DataFrame(matriz, 
                              index=nombres_estrategias_filas,
                              columns=nombres_estrategias_columnas)
            print(df)
        else:
            matriz_j1 = np.zeros((filas, columnas))
            matriz_j2 = np.zeros((filas, columnas))
            for i in range(filas):
                for j in range(columnas):
                    matriz_j1[i][j] = cells_matriz_j1[i][j].value
                    matriz_j2[i][j] = cells_matriz_j2[i][j].value
            
            print("\nMatrices de pagos confirmadas (Suma No Cero):")
            print("\nMatriz Jugador Usuario:")
            df1 = pd.DataFrame(matriz_j1, 
                              index=nombres_estrategias_filas,
                              columns=nombres_estrategias_columnas)
            print(df1)
            print("\nMatriz Jugador Contrincante:")
            df2 = pd.DataFrame(matriz_j2, 
                              index=nombres_estrategias_filas,
                              columns=nombres_estrategias_columnas)
            print(df2)

boton_confirmar.on_click(on_confirmar_click)

display(VBox(nombres_filas))
display(VBox(nombres_columnas))
display(boton_actualizar)
display(output_grid)
display(boton_confirmar, output_matriz)

actualizar_grid()

In [ ]:
print("="*60)
print("ANÁLISIS DE TEORÍA DE LOS JUEGOS")
print("="*60)

if modo_juego == 'suma_cero':
    print("\nMODO: SUMA CERO (Competitivo)")
    df = pd.DataFrame(matriz, index=nombres_estrategias_filas, columns=nombres_estrategias_columnas)
    print("\nMatriz de Pagos del Jugador Usuario:")
    print(df)
else:
    print("\nMODO: SUMA NO CERO (General)")
    df_j1 = pd.DataFrame(matriz_j1, index=nombres_estrategias_filas, columns=nombres_estrategias_columnas)
    df_j2 = pd.DataFrame(matriz_j2, index=nombres_estrategias_filas, columns=nombres_estrategias_columnas)
    print("\nMatriz de Pagos Jugador Usuario:")
    print(df_j1)
    print("\nMatriz de Pagos Jugador Contrincante:")
    print(df_j2)

print("\n" + "="*60)

if modo_juego == 'suma_cero':
    
    print("\n1. TIPO DE JUEGO: SUMA CERO")
    print("-"*60)
    print("✓ Juego estrictamente competitivo")
    print("✓ Lo que gana un jugador lo pierde el otro")
    
    print("\n2. ESTRATEGIAS DOMINANTES")
    print("-"*60)
    
    dominantes_filas = []
    for i in range(filas):
        es_dominante = True
        for k in range(filas):
            if k != i:
                if not all(matriz[i, j] >= matriz[k, j] for j in range(columnas)):
                    es_dominante = False
                    break
        if es_dominante and any(matriz[i, j] > matriz[k, j] for k in range(filas) if k != i for j in range(columnas)):
            dominantes_filas.append(i)
            print(f"✓ Estrategia DOMINANTE para Usuario: {nombres_estrategias_filas[i]}")
    
    if not dominantes_filas:
        print("✗ No hay estrategias estrictamente dominantes para el Jugador Usuario")
    
    dominantes_columnas = []
    for j in range(columnas):
        es_dominante = True
        for k in range(columnas):
            if k != j:
                if not all(matriz[i, j] >= matriz[i, k] for i in range(filas)):
                    es_dominante = False
                    break
        if es_dominante and any(matriz[i, j] > matriz[i, k] for k in range(columnas) if k != j for i in range(filas)):
            dominantes_columnas.append(j)
            print(f"✓ Estrategia DOMINANTE para Contrincante: {nombres_estrategias_columnas[j]}")
    
    if not dominantes_columnas:
        print("✗ No hay estrategias estrictamente dominantes para el Jugador Contrincante")
    
    print("\n3. CRITERIO MAXMIN Y MINMAX")
    print("-"*60)
    
    maxmin_valores = np.min(matriz, axis=1)
    maxmin_idx = np.argmax(maxmin_valores)
    maxmin_valor = maxmin_valores[maxmin_idx]
    
    print(f"MAXMIN (Usuario maximiza su mínimo):")
    print(f"  Estrategia: {nombres_estrategias_filas[maxmin_idx]}")
    print(f"  Valor garantizado: {maxmin_valor}")
    
    minmax_valores = np.max(matriz, axis=0)
    minmax_idx = np.argmin(minmax_valores)
    minmax_valor = minmax_valores[minmax_idx]
    
    print(f"\nMINMAX (Contrincante minimiza el máximo del Usuario):")
    print(f"  Estrategia: {nombres_estrategias_columnas[minmax_idx]}")
    print(f"  Valor del juego: {minmax_valor}")
    
    tiene_punto_silla = False
    if maxmin_valor == minmax_valor:
        print(f"\n✓ PUNTO DE SILLA encontrado en ({nombres_estrategias_filas[maxmin_idx]}, {nombres_estrategias_columnas[minmax_idx]})")
        print(f"  Valor del juego: {maxmin_valor}")
        print(f"  Solución en ESTRATEGIAS PURAS")
        tiene_punto_silla = True
        eq_nash_puro = [(maxmin_idx, minmax_idx)]
    else:
        print(f"\n✗ NO hay punto de silla ({maxmin_valor} ≠ {minmax_valor})")
        print(f"  Se requieren ESTRATEGIAS MIXTAS")
    
    print("\n4. EQUILIBRIOS DE NASH EN ESTRATEGIAS PURAS")
    print("-"*60)
    
    if not tiene_punto_silla:
        eq_nash_puro = []
        for i in range(filas):
            for j in range(columnas):
                es_equilibrio_fila = all(matriz[i, j] >= matriz[k, j] for k in range(filas))
                es_equilibrio_columna = all(matriz[i, j] <= matriz[i, k] for k in range(columnas))
                
                if es_equilibrio_fila and es_equilibrio_columna:
                    eq_nash_puro.append((i, j))
        
        if eq_nash_puro:
            for eq in eq_nash_puro:
                print(f"✓ Equilibrio de Nash: ({nombres_estrategias_filas[eq[0]]}, {nombres_estrategias_columnas[eq[1]]}) = {matriz[eq[0], eq[1]]}")
        else:
            print("✗ No hay equilibrios de Nash en estrategias puras")
    
    print("\n5. ESTRATEGIAS MIXTAS (Programación Lineal)")
    print("-"*60)
    
    if filas == 2 and columnas == 2:
        a11, a12 = matriz[0, 0], matriz[0, 1]
        a21, a22 = matriz[1, 0], matriz[1, 1]
        
        denom = (a11 - a21 - a12 + a22)
        if abs(denom) > 1e-10:
            p1 = (a22 - a12) / denom
            p2 = 1 - p1
            
            if 0 <= p1 <= 1:
                q1 = (a22 - a21) / denom
                q2 = 1 - q1
                
                if 0 <= q1 <= 1:
                    valor_juego = (a11*a22 - a12*a21) / denom
                    
                    print(f"Estrategia Mixta Usuario:")
                    print(f"  {nombres_estrategias_filas[0]}: {p1:.4f}")
                    print(f"  {nombres_estrategias_filas[1]}: {p2:.4f}")
                    
                    print(f"\nEstrategia Mixta Contrincante:")
                    print(f"  {nombres_estrategias_columnas[0]}: {q1:.4f}")
                    print(f"  {nombres_estrategias_columnas[1]}: {q2:.4f}")
                    
                    print(f"\nValor del juego: {valor_juego:.4f}")
                else:
                    print("Solución en estrategias puras (q fuera de rango)")
            else:
                print("Solución en estrategias puras (p fuera de rango)")
        else:
            print("Matriz degenerada, múltiples soluciones posibles")
    else:
        try:
            c = [-1] + [0] * filas
            A_ub = []
            for j in range(columnas):
                fila = [-1] + [-matriz[i, j] for i in range(filas)]
                A_ub.append(fila)
            b_ub = [0] * columnas
            A_eq = [[0] + [1] * filas]
            b_eq = [1]
            bounds = [(None, None)] + [(0, 1) for _ in range(filas)]
            
            resultado = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
            
            if resultado.success:
                valor_juego = resultado.x[0]
                prob_usuario = resultado.x[1:]
                
                print(f"Estrategia Mixta Usuario:")
                for i in range(filas):
                    if prob_usuario[i] > 1e-6:
                        print(f"  {nombres_estrategias_filas[i]}: {prob_usuario[i]:.4f}")
                
                print(f"\nValor del juego: {valor_juego:.4f}")
            else:
                print("No se pudo encontrar solución óptima")
        except:
            print("Error al calcular estrategias mixtas para matriz de tamaño mayor")
    
    print("\n" + "="*60)
    print("6. VISUALIZACIÓN")
    print("="*60)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    sns.heatmap(matriz, annot=True, fmt='.2f', cmap='RdYlGn', center=0, 
                xticklabels=nombres_estrategias_columnas, 
                yticklabels=nombres_estrategias_filas,
                cbar_kws={'label': 'Pago Usuario'}, ax=axes[0])
    axes[0].set_title('Matriz de Pagos del Jugador Usuario', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Estrategias Contrincante', fontweight='bold')
    axes[0].set_ylabel('Estrategias Usuario', fontweight='bold')
    
    if eq_nash_puro:
        for eq in eq_nash_puro:
            axes[0].add_patch(plt.Rectangle((eq[1], eq[0]), 1, 1, fill=False, edgecolor='blue', lw=3))
    
    maxmin_visual = np.zeros((filas, columnas))
    for i in range(filas):
        min_val = np.min(matriz[i, :])
        for j in range(columnas):
            if matriz[i, j] == min_val:
                maxmin_visual[i, j] = 1
    
    minmax_visual = np.zeros((filas, columnas))
    for j in range(columnas):
        max_val = np.max(matriz[:, j])
        for i in range(filas):
            if matriz[i, j] == max_val:
                minmax_visual[i, j] = 1
    
    criterios = maxmin_visual + minmax_visual
    sns.heatmap(criterios, annot=True, fmt='.0f', cmap='Blues', 
                xticklabels=nombres_estrategias_columnas,
                yticklabels=nombres_estrategias_filas,
                cbar_kws={'label': 'Puntuación'}, ax=axes[1])
    axes[1].set_title('Análisis MaxMin y MinMax\n(2 = Punto de Silla)', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Estrategias Contrincante', fontweight='bold')
    axes[1].set_ylabel('Estrategias Usuario', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

else:
    
    print("\n1. TIPO DE JUEGO: SUMA NO CERO (General)")
    print("-"*60)
    print("✓ Juego bimatricial con pagos independientes")
    print("✓ Cada jugador tiene su propia matriz de pagos")
    
    print("\n2. ESTRATEGIAS DOMINANTES")
    print("-"*60)
    
    print("Jugador Usuario:")
    dominantes_j1 = []
    for i in range(filas):
        es_dominante = True
        for k in range(filas):
            if k != i:
                if not all(matriz_j1[i, j] >= matriz_j1[k, j] for j in range(columnas)):
                    es_dominante = False
                    break
        if es_dominante and any(matriz_j1[i, j] > matriz_j1[k, j] for k in range(filas) if k != i for j in range(columnas)):
            dominantes_j1.append(i)
            print(f"  ✓ Estrategia DOMINANTE: {nombres_estrategias_filas[i]}")
    
    if not dominantes_j1:
        print("  ✗ No hay estrategias estrictamente dominantes")
    
    print("\nJugador Contrincante:")
    dominantes_j2 = []
    for j in range(columnas):
        es_dominante = True
        for k in range(columnas):
            if k != j:
                if not all(matriz_j2[i, j] >= matriz_j2[i, k] for i in range(filas)):
                    es_dominante = False
                    break
        if es_dominante and any(matriz_j2[i, j] > matriz_j2[i, k] for k in range(columnas) if k != j for i in range(filas)):
            dominantes_j2.append(j)
            print(f"  ✓ Estrategia DOMINANTE: {nombres_estrategias_columnas[j]}")
    
    if not dominantes_j2:
        print("  ✗ No hay estrategias estrictamente dominantes")
    
    print("\n3. EQUILIBRIOS DE NASH EN ESTRATEGIAS PURAS")
    print("-"*60)
    
    eq_nash = []
    for i in range(filas):
        for j in range(columnas):
            es_mejor_resp_j1 = all(matriz_j1[i, j] >= matriz_j1[k, j] for k in range(filas))
            es_mejor_resp_j2 = all(matriz_j2[i, j] >= matriz_j2[i, k] for k in range(columnas))
            
            if es_mejor_resp_j1 and es_mejor_resp_j2:
                eq_nash.append((i, j))
    
    if eq_nash:
        for eq in eq_nash:
            print(f"✓ Equilibrio de Nash: ({nombres_estrategias_filas[eq[0]]}, {nombres_estrategias_columnas[eq[1]]})")
            print(f"  Pagos: Usuario={matriz_j1[eq[0], eq[1]]:.2f}, Contrincante={matriz_j2[eq[0], eq[1]]:.2f}")
    else:
        print("✗ No hay equilibrios de Nash en estrategias puras")
    
    print("\n4. ESTRATEGIAS MAXMIN")
    print("-"*60)
    
    print("Jugador Usuario:")
    maxmin_j1 = np.min(matriz_j1, axis=1)
    maxmin_idx_j1 = np.argmax(maxmin_j1)
    print(f"  Estrategia MaxMin: {nombres_estrategias_filas[maxmin_idx_j1]}")
    print(f"  Valor garantizado: {maxmin_j1[maxmin_idx_j1]:.2f}")
    
    print("\nJugador Contrincante:")
    maxmin_j2 = np.min(matriz_j2, axis=0)
    maxmin_idx_j2 = np.argmax(maxmin_j2)
    print(f"  Estrategia MaxMin: {nombres_estrategias_columnas[maxmin_idx_j2]}")
    print(f"  Valor garantizado: {maxmin_j2[maxmin_idx_j2]:.2f}")
    
    print("\n" + "="*60)
    print("5. VISUALIZACIÓN")
    print("="*60)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    sns.heatmap(matriz_j1, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
                xticklabels=nombres_estrategias_columnas,
                yticklabels=nombres_estrategias_filas,
                cbar_kws={'label': 'Pago'}, ax=axes[0, 0])
    axes[0, 0].set_title('Matriz de Pagos - Jugador Usuario', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Estrategias Contrincante', fontweight='bold')
    axes[0, 0].set_ylabel('Estrategias Usuario', fontweight='bold')
    
    sns.heatmap(matriz_j2, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
                xticklabels=nombres_estrategias_columnas,
                yticklabels=nombres_estrategias_filas,
                cbar_kws={'label': 'Pago'}, ax=axes[0, 1])
    axes[0, 1].set_title('Matriz de Pagos - Jugador Contrincante', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Estrategias Contrincante', fontweight='bold')
    axes[0, 1].set_ylabel('Estrategias Usuario', fontweight='bold')
    
    if eq_nash:
        for eq in eq_nash:
            axes[0, 0].add_patch(plt.Rectangle((eq[1], eq[0]), 1, 1, fill=False, edgecolor='blue', lw=3))
            axes[0, 1].add_patch(plt.Rectangle((eq[1], eq[0]), 1, 1, fill=False, edgecolor='blue', lw=3))
    
    matriz_suma = matriz_j1 + matriz_j2
    sns.heatmap(matriz_suma, annot=True, fmt='.2f', cmap='viridis',
                xticklabels=nombres_estrategias_columnas,
                yticklabels=nombres_estrategias_filas,
                cbar_kws={'label': 'Suma Total'}, ax=axes[1, 0])
    axes[1, 0].set_title('Suma de Pagos (Eficiencia Social)', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Estrategias Contrincante', fontweight='bold')
    axes[1, 0].set_ylabel('Estrategias Usuario', fontweight='bold')
    
    mejor_resp_visual = np.zeros((filas, columnas))
    for j in range(columnas):
        mejor_i = np.argmax(matriz_j1[:, j])
        mejor_resp_visual[mejor_i, j] += 1
    for i in range(filas):
        mejor_j = np.argmax(matriz_j2[i, :])
        mejor_resp_visual[i, mejor_j] += 1
    
    sns.heatmap(mejor_resp_visual, annot=True, fmt='.0f', cmap='Blues',
                xticklabels=nombres_estrategias_columnas,
                yticklabels=nombres_estrategias_filas,
                cbar_kws={'label': 'Puntuación'}, ax=axes[1, 1])
    axes[1, 1].set_title('Análisis de Mejores Respuestas\n(2 = Equilibrio de Nash)', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Estrategias Contrincante', fontweight='bold')
    axes[1, 1].set_ylabel('Estrategias Usuario', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

print("\n✓ Análisis completo finalizado")